In [6]:
import numpy as np
import pandas as pd
import plotly.graph_objs as go
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam

In [7]:
def prepare_data(data, time_steps=1):
    """
    Prepare data for LSTM by creating sliding window sequences.
    
    Args:
        data (np.array): Input data series
        time_steps (int): Number of previous time steps to use for prediction
    
    Returns:
        X (np.array): Input sequences
        y (np.array): Target values
    """
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i:(i + time_steps)])
        y.append(data[i + time_steps])
    return np.array(X), np.array(y)

def create_lstm_model(input_shape, units=50):
    """
    Create LSTM model for time series forecasting.
    
    Args:
        input_shape (tuple): Shape of input data
        units (int): Number of LSTM units
    
    Returns:
        model (keras.Model): Compiled LSTM model
    """
    model = Sequential([
        LSTM(units, activation='relu', input_shape=input_shape),
        Dense(1)
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    return model

def train_lstm_model(X_train, y_train, time_steps, epochs=50, batch_size=32):
    """
    Train LSTM model.
    
    Args:
        X_train (np.array): Training input sequences
        y_train (np.array): Training target values
        time_steps (int): Number of previous time steps used
        epochs (int): Number of training epochs
        batch_size (int): Batch size for training
    
    Returns:
        model (keras.Model): Trained LSTM model
    """
    model = create_lstm_model(input_shape=(time_steps, 1), units=50)
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0)
    return model

def forecast_with_lstm(df, column='Close', test_size=0.2, time_steps=10, forecast_days=30):
    """
    Perform LSTM forecasting on time series data.
    
    Args:
        df (pd.DataFrame): Input DataFrame with time series data
        column (str): Column to forecast
        test_size (float): Proportion of data to use for testing
        time_steps (int): Number of previous time steps to use for prediction
        forecast_days (int): Number of days to forecast
    
    Returns:
        dict: Forecasting results including predictions, model performance metrics
    """
    # Prepare data
    data = df[column].values.reshape(-1, 1)
    
    # Scale the data
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(data)
    
    # Prepare sequences
    X, y = prepare_data(scaled_data, time_steps)
    
    # Split data
    split = int(len(X) * (1 - test_size))
    X_train, X_test = X[:split], X[split:]
    y_train, y_test = y[:split], y[split:]
    
    # Train model
    model = train_lstm_model(X_train, y_train, time_steps)
    
    # Predict and inverse transform
    train_predict = model.predict(X_train)
    test_predict = model.predict(X_test)
    
    train_predict = scaler.inverse_transform(train_predict)
    y_train_inv = scaler.inverse_transform(y_train)
    test_predict = scaler.inverse_transform(test_predict)
    y_test_inv = scaler.inverse_transform(y_test)
    
    # Compute performance metrics
    train_mse = mean_squared_error(y_train_inv, train_predict)
    test_mse = mean_squared_error(y_test_inv, test_predict)
    train_mae = mean_absolute_error(y_train_inv, train_predict)
    test_mae = mean_absolute_error(y_test_inv, test_predict)
    
    # Future forecasting
    last_sequence = scaled_data[-time_steps:].reshape((1, time_steps, 1))
    future_predictions = []
    
    for _ in range(forecast_days):
        next_pred = model.predict(last_sequence)[0]
        future_predictions.append(next_pred[0])
        last_sequence = np.roll(last_sequence, -1)
        last_sequence[0, -1, 0] = next_pred[0]
    
    future_predictions = scaler.inverse_transform(np.array(future_predictions).reshape(-1, 1))
    
    # Plotly Interactive Visualization
    def create_plotly_visualization(original_data, test_predict, future_predictions, df):
        """
        Create an interactive Plotly visualization of the forecast.
        
        Args:
            original_data (np.array): Original time series data
            test_predict (np.array): Test predictions
            future_predictions (np.array): Future forecast predictions
            df (pd.DataFrame): Original DataFrame
        
        Returns:
            plotly.graph_objs.Figure: Interactive visualization
        """
        # Prepare dates for plotting
        original_dates = df.index
        test_start_date = original_dates[-len(test_predict):]
        future_dates = pd.date_range(start=original_dates[-1], periods=len(future_predictions)+1)[1:]
        
        # Create figure
        fig = go.Figure()
        
        # Original data trace
        fig.add_trace(go.Scatter(
            x=original_dates, 
            y=original_data.flatten(), 
            mode='lines', 
            name='Original Data', 
            line=dict(color='blue')
        ))
        
        # Test predictions trace
        fig.add_trace(go.Scatter(
            x=test_start_date, 
            y=test_predict.flatten(), 
            mode='lines', 
            name='Test Predictions', 
            line=dict(color='red', dash='dot')
        ))
        
        # Future predictions trace
        fig.add_trace(go.Scatter(
            x=future_dates, 
            y=future_predictions.flatten(), 
            mode='lines', 
            name='Future Forecasts', 
            line=dict(color='green', dash='dash')
        ))
        
        # Update layout
        fig.update_layout(
            title='LSTM Time Series Forecast',
            xaxis_title='Date',
            yaxis_title='Close Price',
            hovermode='x unified',
            legend_title_text='Data Types'
        )
        
        return fig
    
    # Generate and show Plotly figure
    plotly_fig = create_plotly_visualization(data, test_predict, future_predictions, df)
    plotly_fig.show()
    
    return {
        'future_predictions': future_predictions,
        'train_mse': train_mse,
        'test_mse': test_mse,
        'train_mae': train_mae,
        'test_mae': test_mae,
        'plotly_figure': plotly_fig
    }

# Example usage
#if __name__ == "__main__":
    # Load your time series data here
    # df = pd.read_csv('your_time_series_data.csv', parse_dates=['Date'], index_col='Date')
    
    # Uncomment and modify the following line with your actual data
    # results = forecast_with_lstm(df, forecast_days=30)
#    pass

In [10]:
results['future_predictions']

array([[38.556793],
       [38.294674],
       [37.972336],
       [37.642433],
       [37.276375],
       [36.904716],
       [36.522392]], dtype=float32)

In [8]:
# Load your time series data here
df = pd.read_csv('../dados/raw/PETR4.SA.csv', parse_dates=['Date'], index_col='Date')

# Uncomment and modify the following line with your actual data
results = forecast_with_lstm(df, forecast_days=7)

1/1 [==============================] - 0s 88ms/step
